# Setup

In [57]:
% pip install imageio
% pip install pyvirtualdisplay
% pip install tf-agents
% pip install pyglet
% pip install matplotlib
% pip install tensorflow

UsageError: Line magic function `%` not found.


In [ ]:
import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

# Seed for PPO actor network
import tensorflow_probability as tfp

# PPO Agent
from tf_agents.agents.ppo import ppo_agent
from tf_agents.agents.ppo import ppo_actor_network
from tf_agents.networks import value_network
from tf_agents.networks import actor_distribution_network
from tf_agents.agents.ppo import ppo_clip_agent

from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step as ts

from tf_agents.networks import sequential

# old agent
from tf_agents.agents.categorical_dqn import categorical_dqn_agent
from tf_agents.networks import categorical_q_network

from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

# Set up a virtual display for rendering OpenAI gym environments.
# display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

# Hyperparameters

In [75]:

# register(
#     id='CartPole-v1',
#     entry_point='gym.envs.classic_control:CartPoleEnv',
#     max_episode_steps=500,
#     reward_threshold=475.0,
# )

env_name = "CartPole-v1"  # @param {type:"string"}
# env_name = "Acrobot-v1"
num_iterations = 5000  # @param {type:"integer"}

initial_collect_steps = 1000  # @param {type:"integer"} 
collect_steps_per_iteration = 100  # @param {type:"integer"}
replay_buffer_capacity = 100000  # @param {type:"integer"}

fc_layer_params = (64, 64)

batch_size = 128  # @param {type:"integer"}
learning_rate = 3e-4  # @param {type:"number"}
gamma = 0.99




# Environment

In [76]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

observation_tensor_spec = tensor_spec.from_spec(train_env.observation_spec())
action_tensor_spec = tensor_spec.from_spec(train_env.action_spec())
time_step_tensor_spec = tensor_spec.from_spec(train_env.time_step_spec())

In [77]:
print('Observation: {0}'.format(observation_tensor_spec))
print('Action: {0}'.format(action_tensor_spec))
print('TimeStep: {0}'.format(time_step_tensor_spec))

Observation: BoundedTensorSpec(shape=(4,), dtype=tf.float32, name='observation', minimum=array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38],
      dtype=float32), maximum=array([4.8000002e+00, 3.4028235e+38, 4.1887903e-01, 3.4028235e+38],
      dtype=float32))
Action: BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(1))
TimeStep: TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': BoundedTensorSpec(shape=(4,), dtype=tf.float32, name='observation', minimum=array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38],
      dtype=float32), maximum=array([4.8000002e+00, 3.4028235e+38, 4.1887903e-01, 3.4028235e+38],
      dtype=float32)),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})


# Agent

Netowork

In [149]:
# actor_fc_layers = (64, 64)
# value_fc_layers = (64, 64)
# 
# actor_net = actor_distribution_network.ActorDistributionNetwork(
#     observation_tensor_spec,
#     action_tensor_spec,
#     fc_layer_params=actor_fc_layers,
#     activation_fn=tf.keras.activations.tanh,
#     seed_stream_class=tfp.util.SeedStream(seed=None, salt='tf_agents_sequential_layers')
# )
# 
# value_net = value_network.ValueNetwork(
#     observation_tensor_spec,
#     fc_layer_params=value_fc_layers,
#     activation_fn=tf.keras.activations.tanh,
#     kernel_initializer=tf.keras.initializers.Orthogonal()
# )

# norm = tf.keras.layers.LayerNormalization(axis=None)


actor_fc_layers = (64, 64)
value_fc_layers = (64, 64)


actor_net = actor_distribution_network.ActorDistributionNetwork(
    input_tensor_spec=observation_tensor_spec,
    output_tensor_spec=action_tensor_spec,
    fc_layer_params=actor_fc_layers,
    activation_fn=tf.keras.activations.tanh,
    kernel_initializer=tf.keras.initializers.Orthogonal(seed=1),
    seed_stream_class=tfp.util.SeedStream(seed=None, salt='tf_agents_sequential_layers')
)

value_net = value_network.ValueNetwork(
    observation_tensor_spec,
    fc_layer_params=value_fc_layers,
    activation_fn=tf.keras.activations.tanh,
    kernel_initializer=tf.keras.initializers.Orthogonal()
)

In [150]:
print(observation_tensor_spec)
print(action_tensor_spec)
print(time_step_tensor_spec)

BoundedTensorSpec(shape=(4,), dtype=tf.float32, name='observation', minimum=array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38],
      dtype=float32), maximum=array([4.8000002e+00, 3.4028235e+38, 4.1887903e-01, 3.4028235e+38],
      dtype=float32))
BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(1))
TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': BoundedTensorSpec(shape=(4,), dtype=tf.float32, name='observation', minimum=array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38],
      dtype=float32), maximum=array([4.8000002e+00, 3.4028235e+38, 4.1887903e-01, 3.4028235e+38],
      dtype=float32)),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})


In [151]:
# learning_rate = 3e-4
# optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
# epsilon = 0.2
# discount = 0.99
# gae_lambda = 0.95
# train_step_counter = tf.Variable(0)
# 
# agent_ppo = ppo_clip_agent.PPOClipAgent(
#     time_step_spec=time_step_tensor_spec,
#     action_spec=action_tensor_spec,
#     actor_net=actor_net,
#     value_net=value_net,
#     optimizer=optimizer,
#     use_gae=True,
#     lambda_value=gae_lambda,
#     importance_ratio_clipping=epsilon,
#     # log_prob_clipping=epsilon,
#     # gradient_clipping=epsilon,
#     # value_clipping=epsilon,
#     train_step_counter=train_step_counter,
#     normalize_observations=True,
#     normalize_rewards=True
# )
# agent_ppo.initialize()


learning_rate = 3e-4
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
epsilon = 0.2
discount = 0.99
gae_lambda = 0.95
train_step_counter = tf.Variable(0)

agent_ppo = ppo_agent.PPOAgent(
    time_step_spec=time_step_tensor_spec,
    action_spec=action_tensor_spec,
    optimizer=optimizer,
    actor_net=actor_net,
    value_net=value_net,
    greedy_eval=False,
    lambda_value=gae_lambda,
    discount_factor=discount,
    importance_ratio_clipping=epsilon,
    num_epochs=10,
    # use_gae=True,
    # log_prob_clipping=epsilon,
    # gradient_clipping=epsilon,
    # value_clipping=epsilon,
    train_step_counter=train_step_counter,
)
agent_ppo.initialize()


# Metrics and Evaluation

In [152]:
train_env.current_time_step()

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 4), dtype=float32, numpy=
array([[ 0.01215199, -0.19767444, -0.01313889,  0.33657083]],
      dtype=float32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>})

In [153]:
def compute_avg_return(environment, policy, num_episodes=10):
    total_return = 0.0
    for _ in range(num_episodes):

        time_step = environment.reset()
        episode_return = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
        total_return += episode_return
    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]


random_policy = random_tf_policy.RandomTFPolicy(
    time_step_tensor_spec,
    action_tensor_spec,
    info_spec=time_step_tensor_spec)


def evaluate_policy(policy):
    for _ in range(1):
        avg = compute_avg_return(eval_env, policy, num_eval_episodes)
        print(avg)


# evaluate_policy(random_policy)

# evaluate_policy(agent_ppo.policy)

# evaluate_policy(agent_ppo.collect_policy)


# Data Collection

In [154]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    agent_ppo.collect_policy.trajectory_spec,
    batch_size=1,
    max_length=10000)

def collect_step(environment, policy):
    time_step = environment.current_time_step()
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step.action)
    traj = trajectory.from_transition(time_step, action_step, next_time_step)
    replay_buffer.add_batch(traj)
    # print(next_time_step.reward)


for _ in range(initial_collect_steps):
    collect_step(train_env, agent_ppo.collect_policy)

# This loop is so common in RL, that we provide standard implementations of
# these. For more details see the drivers module.

# Dataset generates trajectories with shape [BxTx...] where
# T = n_step_update + 1.
n_step_update = 2
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, sample_batch_size=batch_size,
    num_steps=n_step_update + 1).prefetch(batch_size)

iterator = iter(dataset)

# print(f'TimeStep: {train_env.time_step_spec()}')
# print(f'Action: {train_env.action_spec()}')
# print(f'Agent: {agent_ppo.collect_policy.time_step_spec}')

# Training the Agent

In [155]:
try:
    % % time
except:
    pass

num_iterations = 1000
collect_steps_per_iteration = 1
eval_interval = 10
num_eval_episodes = 1

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent_ppo.train = common.function(agent_ppo.train)

# Reset the train step
agent_ppo.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent_ppo.policy, num_eval_episodes)
returns = [avg_return]
returns

for _ in range(num_iterations-1):
    # Collect a few steps using collect_policy and save to the replay buffer.
    for _ in range(collect_steps_per_iteration):
        collect_step(train_env, agent_ppo.collect_policy)

    experience, unused_info = next(iterator)
    train_loss = agent_ppo.train(experience).loss

    step = agent_ppo.train_step_counter.numpy()

    if step % eval_interval == 0:
        avg_return = compute_avg_return(eval_env, agent_ppo.policy, num_eval_episodes)
        print('step = {0}: loss = {1}  AVG RETURN: {2}'.format(step, train_loss, avg_return))
        returns.append(avg_return)


step = 10: loss = 73.86439514160156  AVG RETURN: 37.0
step = 20: loss = 13.35525131225586  AVG RETURN: 15.0
step = 30: loss = 15.082578659057617  AVG RETURN: 85.0
step = 40: loss = 15.678494453430176  AVG RETURN: 15.0
step = 50: loss = 15.460611343383789  AVG RETURN: 13.0
step = 60: loss = 15.94218635559082  AVG RETURN: 18.0
step = 70: loss = 15.926673889160156  AVG RETURN: 14.0
step = 80: loss = 16.701637268066406  AVG RETURN: 20.0
step = 90: loss = 16.844234466552734  AVG RETURN: 10.0
step = 100: loss = 16.66084861755371  AVG RETURN: 34.0
step = 110: loss = 15.974098205566406  AVG RETURN: 32.0
step = 120: loss = 16.410736083984375  AVG RETURN: 22.0
step = 130: loss = 16.347003936767578  AVG RETURN: 30.0
step = 140: loss = 16.303861618041992  AVG RETURN: 27.0
step = 150: loss = 15.960721015930176  AVG RETURN: 16.0
step = 160: loss = 15.748817443847656  AVG RETURN: 12.0
step = 170: loss = 15.414562225341797  AVG RETURN: 19.0
step = 180: loss = 15.612872123718262  AVG RETURN: 13.0
step 

# Visualization

In [156]:
len(returns)

1000

In [1]:
# steps = range(0, num_iterations + 1, int(num_iterations/len(returns)) )
steps = range(0, num_iterations , 2 )

# steps = range(0, num_iterations + 1, 10)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim(top=max(returns) + 10)

NameError: name 'num_iterations' is not defined